<span type="title">Hibernate without JPA</span> | <span type="update">2018-10-24</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">本章主要介绍 Hibernate 的基于 XML 配置方式的传统使用方式，而没有涉及基于注解的 JPA API 的使用方式。本文首先介绍了 Object/Mapping 的基本概念，接着从对象关系映射的角度介绍了 Hibernate 的总体工作流程，在其中穿插讲解了 cfg.xml 的基础 Hiberante 配置，hbm.xml 的对象映射文件配置（包括时间日期映射和大文件映射），基本的 Hibernate API 使用（SF、Session、Transaction），结合 JUnit 和 绑定线程的 Hibernate 使用方式。</p><p class="card-text">在第二部分，主要介绍了 Hibernate 的基础概念，包括 Session 缓存（flush、refresh、clear）, 几种不同的对象状态（游离、临时、持久化、删除对象），Session API 的大部分操纵状态的方法（save、saveOrUpdate、persist、close、evict、clear、update、get、load、delete 等）。</p><p class="card-text">在第三部分，介绍了 Hibernate 对象关系映射的高级部分，包括组成、单向一对多、双向一对多、基于外键的一对一、基于主键的一对一、单向和双向的多对多、subclass、united-subclass、joined-subclass 三种继承关系的映射。</p><p class="card-text">在第四部分介绍了 Hibernate 的查询策略以节约开销，包括 fetch、lazy 在类和集合上的使用。同时介绍了 Hibernate 的基于 HQL 的查询方式，比如查询、更新、投影、分组、命名、分页、左外连接、内连接，也介绍了基于 SQL 的查询方法和批处理方式。</p><p class="card-text">在第五部分，介绍了 Hibernate 二级缓存的配置和使用，包括对于类、集合、查询使用二级缓存的方式。</p></span>

# 对象持久化 ORM

Hibernate 是一个对象持久化框架，广义上的持久化指的是 OOP 对象和 数据库行列之间的交互，包括 增加、删除、更改、查找（CRUD）。Hibernate 特有一个叫做“加载”的概念，这是指根据 OID，将一个对象从数据库加载到内存中的操作。其中 OID 指的是，对象标识，OO 对象的属性对应数据库的列，OO 的每个对象对应数据库的行，这种映射称之为 ORM(Object/Relation Map)。为了满足对象和 OO 之间的对应，我们所需要一个指定关联，这个关联叫做 Object Identifier。

ORM 不仅仅需要有 OID 的行对应，还需要有对象的列对应，列对应属性。在 DBUtils or JdbcTemplate 中，我们约定好了一种简单的映射规则，其中属性全小写作为数据库列名标识，这样就完成了对应。但对于实际情况，一个对象可能包含非基本类型的其它类型属性，这时候就需要使用多张表完成此对象的映射，在这种情况下，我们常常需要提供一个 .xml 文件完成这种属性和列的手动映射。

ORM 的架构分为三层，上层是业务逻辑，这里包含了域模型，其中包含了 ORM 需要处理的一些问题：对象、属性、关联、继承和多态。中间层是 ORM 提供的 API，以及其具体实现。Java 有官方的 API，叫做 JPA，Hibernate 是 JPA 的实现的一个超集，支持 xml 和注解两种方式定义域模型和映射关系。下层是数据库层，这里负责维护关系数据模型，包括表、字段、索引、主键和外键。业务逻辑层和数据库层之间通过 xml 或者注解联系起来，完成类和数据库表之间的映射，以支持 CRUD 的操作。

ORM 的业务逻辑域模型被封装成 Data Access Object 对象，负责进行 CRUD 的操作。DAO 可以由 JDBC，DBUtiles，JdbcTemplate 来实现，也可以由 Hibernate，MyBatis 来实现。前者称之为工具，按照约定来进行简单映射，后者称之为框架，其中 Hibernate 使用 xml 文件或者注解进行域模型和数据模型的全自动关联，MyBaits 使用 xml 文件或者 Mapper 接口配合 SQL 进行域模型和数据模型的手动关联。

# Hibernate 工作流程

为了使用 Hibernate 创建 “Hello， World”，最基本的，我们需要有域模型内的 Java 持久化对象，此外还要指定此对象的 OID，属性和数据表列的对应，除此之外，我们还要指定数据库的登录和配置信息，这样，Hibernate 就可以自动在 Java 对象和 数据库表之间建立联系。在整个过程中，我们不需要编写 SQL，预先提供数据表，整个配置完全自动化。其中映射文件，一般以 `.hbm.xml` 结尾，Hibernate 配置文件，以 `hibernate.cfg.xml` 作为名称。

之后，我们就可以通过在 Data Access Object 中通过操作 Hibernate 提供的 API，或者 JPA 标准的 API 完成数据 CRUD 了。

## Hiberante 配置文件

一个基本的 Hibernate.cfg.xml 文件的配置如下, 其中主要包含 session-factory 定义的创建一个连接所需要的配置信息：

- connection 类别的有 username 用户名、password 密码、url 数据库及地址、driver_class 数据库驱动、数据库方言、isolation 数据库隔离级别。
- debug 类别的有 show_sql 显示SQL语句，format_sql 格式化SQL语句显示，hbm2ddl.auto 表的一些操作：总是新建、更新现有、用完即删
- c3p0 数据源的可选配置有 max_size 最大连接, min_size 最小连接, acquire_increment 每次递增，max_statements 最大会话，timeout 检测超时关闭标准，idle_test_period 多久检测一次超时
- jdbc 类别的有：fatch/batch_size 每次查询多少数据、每次批处理执行多少数据（均对 Oracle 有效）
- current_session_class 标签： 基于什么样的方式获取 session 类
- mapping 标签：定义域模型和数据模型关联的配置文件。

```xml
<?xml version='1.0' encoding='utf-8'?>
<!DOCTYPE hibernate-configuration PUBLIC "-//Hibernate/Hibernate Configuration DTD//EN" "http://www.hibernate.org/dtd/hibernate-configuration-3.0.dtd">
<hibernate-configuration>
  <session-factory>
    <!--提供基本的 hibernate 对象和数据库交互时的参数，比如用户名、密码、地址、驱动
    是否在控制台显示SQL，是否自动创建表-->
    <property name="connection.username">corkine</property>
    <property name="connection.password">mi960032</property>
    <property name="connection.driver_class">com.mysql.jdbc.Driver</property>
    <property name="connection.url">jdbc:mysql://localhost:3306/orm</property>
    <!--影响 refresh 能否获取最新的数据，隔离级别四种：1 读未提交 2 读已提交 4 可重复读 8 序列化-->
    <property name="connection.isolation">2</property>

    <property name="show_sql">true</property>
    <property name="format_sql">true</property>
    <!--常见的hbm2ddl.auto的选择： update（更新表，不存在则创建，如果表结构不同，则更新表结构）、create（每次均创建新表）、
    create-drop（用完即删），validate（如果表结构不同或者不存在表，则抛出异常）-->
    <property name="hbm2ddl.auto">update</property>


    <!--配置 datasource 数据源-->
    <property name="c3p0.max_size">1</property>
    <property name="c3p0.min_size">1</property>
    <property name="c3p0.acquire_increment">1</property>
    <property name="c3p0.max_statements">100</property>
    <!--一个链接多长时间没有用，超时设置为 timeout、idel_test 为多长时间检测一次-->
    <property name="c3p0.timeout">1000</property>
    <property name="c3p0.idle_test_period">1000</property>

    <!--配置管理 Session 的方式: 基于 thread-->
    <property name="current_session_context_class">thread</property>

    <!--配置 oracle 的 fetch-size, 每次取回的 sql 行数 batch_size 每次批量更新的 sql 语句数
    注意，此配置仅对 oracle 数据库有效。-->
    <property name="hibernate.jdbc.fetch_size">100</property>
    <property name="hibernate.jdbc.batch_size">30</property>
    <!--使用路径，而不是包名。提供对象映射设置文件-->
    <mapping resource="com/mazhangjing/hibernate/News.hbm.xml" />
    <mapping resource="com/mazhangjing/prohib/Worker.hbm.xml" />
    <mapping resource="com/mazhangjing/hql/Department.hbm.xml" />
  </session-factory>
</hibernate-configuration>
```

## 数据对象及 Hibernate 数据映射

一个持久化 Java 类主要信息，注意，这里 Date 为 java.util.Date，此处的 Bolb 为 java.sql.Blob。省略了 get/set 方法以及无参构造器。

```java
public class News {
    private Integer id;
    private String title;
    private String author;
    private Date date;
    private String desc;
    private String content;
    private Blob image;
    ...
}
```

POJO必须是非 final 类，因为 Hibernate 会使用代理。要提供一个无参构造器，以供通过反射进行对象的获取。此外，最好提供一个标识属性，以映射数据库表的主键字段。并且，最好重写 equals 和 hashCode 方法以进行唯一性隔离。

通过代理和反射创建对象后的调用过程 :

- 通过 .cfg.xml（或者 .properties 文件，或者指定路径下的xml或者prop文件）的设置连接数据库
- 通过提供 .hbm.xml 的对象映射文件来映射 Bean 属性和 SQL表、列、属性
- 通过 setter 和 getter 进行持久化对象和数据库字段的交互
- 通过 ORM API 提供SQL语句进行查询、写入表、更新表

一个非常简单的类-数据库映射文件：

```xml
<hibernate-mapping package="com.mazhangjing.hibernate">
    <class name="News" table="Daily" select-before-update="false" dynamic-update="true">
        <id name="id" type="java.lang.Integer" unsaved-value="none">
            <column name="ID" />
            <generator class="native"/>
        </id>
        <property name="title" type="string" column="TITLE" update="false" unique="true" index="news_index"/>
        <property name="author" type="java.lang.String" column="AUTHOR" index="news_index" />
        <property name="date" type="time" >
            <column name="DATE" />
        </property>
        <property name="desc" type="string" formula="(select concat(author, ':', title) from daily n where n.id = id)" />
        <property name="content" type="string">
            <column name="content" sql-type="mediumtext" />
        </property>
        <property name="image" type="blob">
            <column name="image" sql-type="mediumblob" />
        </property>
    </class>
</hibernate-mapping>
```

**class**

在这个映射文件中，需要提供 hibernate-mapping 标签，其中可以包含数个 class 类标签，package 属性可以定义这些 class 共有的 package，这样，class 中只用指定类名即可，而不用指定全类名。

class 标签需要提供 Java 持久化类的一些映射信息，包括此类对应的数据库表，因此，需要提供 name、table 这两个属性。class 标签内的其余属性也拿过来设置类和表映射的信息，比如动态更新，谨慎更新等。

- dynamic-update 只更新修改的字段，而不更新没有包含的字段
- dynamic-insert 同上
- select-before-update 更新前查询，以确定是否需要更新，影响性能

**id**

id 标签规定了 Object identifier，需要在 name 指定 Java 属性，在 type 指定转换类型，此转换类型将对接两边的类型，进行自动转换，此标签其余属性可定义主键相关设置。推荐使用代理主键（没有任何业务含义的值），整型（节约查询资源）。

- unsaved-value saveOrUpdate 时判断为 save 而不是 update

generator 主键增长方式：
1. increment hibernate 递增，首先确定此表中最大值，然后max(id)+1，并发时容易出现问题
2. identity 由底层数据库负责生成标识符，必须为 long,int 或者 short 类型的数据
3. sequence 由 sequence 标识符生成器从数据库的指定 param 定义的序列获得一个唯一标识号，再作为主键值（Oracle,DB）
4. hilo Hibernate 由高低算法生成标识符，适合所有数据库系统。（hibernate自己维护一个表和列，读取后立马修改next_high，这样就没有并发问题）
5. native 自动根据数据库能力，选择 identity、sequence或者hilo

**property**

property 标签规定了除了 OID 的其余属性（基本类型）的映射，同样的，需要指定 name（Java属性）、type（转换类型）、column（对应的列的名称）。此外，可以使用 update 限定禁止更新（允许新建和删除），unique 指定唯一，index 指定建立索引。此标签可提供 column 子标签，此子标签 name 属性等同于父标签的 column 属性。为了精确完成映射，可以不选择 type 进行自动转换，而在 column 标签指定 sql-type 指定 SQL 端的标签类型（此类型因数据库不同而不同）。

对于 type，如果不提供，则通过反射获取并直接使用 pojo 类型，并且转换为 hibernate 类型。此处可以为 java 类型或者 hibernate 类型。如果设置为 java 类型，则自动转换为 hibernate 类型。

对于 formula，持久化类的有些属性的值必须在运行时通过计算得到，这些pojo属性并不和表中单独一列对应，使用 formula 处理这种映射对应关系
注意，formula 本质是一个子查询，使用括号括起来注意内查询和外查询的对应关系。有 formula 就不需要提供 column 了。

此外，scale 指定小数的位数，对于 double, float, decimal 等类型有效。length 指定字段长度。

**时间日期类型映射**

其中 Java 内置的时间日期为 util.Date, Calender JDBC 扩展了 sql.Date, Time, TimeStamp， 这三种分别对应 SQL 的 DATE, TIME, TIMESTAMP
(Timestamp包含时间和日期信息)。因为 util.Date 是 sql 三个扩展类型的父类，因此推荐使用 util.Date 以包含所有可能的数据库时间日期类型种类。

在 hbm.xml 中，使用 type 属性将 pojo 的 util.Date 映射为 hibernate 类型 time、date、timestamp。这样的话，数据库就会使用 time\date\timestamp 类型。

注意，必须在 pojo 设置 util.Date 类型，如果设置为 sql.Date 类型，而 xml 设置 time 类型，则会转换出错。

**大对象类型映射**

对于Java 而言， 使用 `String` 表示长字符串， `byte[]` 表示图片或者文件二进制， `sql.Clob sql.Blob` 表示 CLOB（字符串大对象） BLOB（二进制大对象）。但是对于 mysql，其不支持 CLOB， 使用 `text mediumtext longtext` 表示长文本。<u>因此，在持久化中，使用 `String` 而不是 `sql.CLOB` 表示长文本（更方便），使用 `sql.BLOB` 而不是 `byte[]` 表示二进制数据</u>（`byte[]` 需要繁琐的转换进行保存和读写，而 Blob 直接可以通过 hibernate.createXXX 获取 io 流）。

由于指定 type 可能导致不恰当的映射，比如 string - mediumtext/text， 使用 sql-type 来精确指定 sql 列的类型。实际使用中，经常是通过文件的方式保存大二进制数据（图片等），而在 sql 数据库中保存位置的引用，而不是直接存储在数据库中（大Blob影响性能）。
因此，最常用的 pojo 类型是 String，对应的 mysql 类型（sql-type） 是 `mediumtext, largetext`。而 `BLOB` 和 `byte[]` 则很少用。

API 中，使用 Hibernate.getLobCreator(Session).createBlob 来加载一个流，指定流长度，来创建 Bolb 对象。使用 getBinaryStream 直接从 Bolb 中获得输出流写入到 FileOutputStream 中即可。

```java
@Test public void setBlob() throws IOException, SQLException {
    InputStream stream = new FileInputStream("cat.jpg");
    Blob blob = Hibernate.getLobCreator(session).createBlob(stream,stream.available());
    News news = new News(1,"aa","AA",new Date());
    news.setImage(blob);
    session.save(news);
    //注意，不能在 save 前关闭 stream，否则会报错。其次，pojo 使用 blob 类更加方便（方便set和get，方便xml映射hibernate blob
    // sql-type:medium-blob）。
}
@Test public void readBlod() throws IOException, SQLException {
    News news = session.get(News.class,1);
    Blob blob = news.getImage();
    FileOutputStream fos = new FileOutputStream("data_in_sql.jpg");
    InputStream in = blob.getBinaryStream();
    int length; byte[] buffer = new byte[1024];
    while ((length = in.read(buffer)) != -1) {
        fos.write(buffer);
    }
}
```


## Hibernate ORM API

一个基本的 ORM API 操纵流程为, 获取配置文件，根据配置文件创建 SF 对象，根据 SF 创建 Session，此处的 Session 类似于 JDBC 的 Connection，负责管理每次数据库调用。创建完 Session 后，获取 Transaction，之后执行操作，基本的操作API为 get，获取指定主键的指定类型的对象，save 保存一个对象到对应映射的数据库表中。然后提交事务、关闭 Session、关闭 SF。

```java
//关联cfg.xml文件
Configuration configuration = new Configuration().configure("hibernate.cfg.xml");
//创建sessionFactory对象
SessionFactory sessionFactory = configuration.buildSessionFactory();
//创建Session
Session session = sessionFactory.openSession();
//开启事务
Transaction transaction = session.beginTransaction();
//执行保存操作
/*News news = session.get(News.class,1);
System.out.println(news);*/
News news = new News(1,"Java","Marvin",new Date());
session.save(news);
//提交事务
transaction.commit();
//关闭Session
session.close();
//关闭SessionFactory
sessionFactory.close();
```

**Configuration API：**

读取 hibernate 配置文件，cfg.xml（调用.configure()方法） 或者 hibernate.properties（不调用.configure()的话）

**SessionFactory API:**

创建Session的工厂，一旦建造完毕，则赋予特定配置信息，比如对象映射、各项数据验证信息。由于创建很消耗资源，因此一个
项目只有一个。从此工厂创建 session。在新版本中的 SF 使用如下方法开启(通过 MS build Mdata，然后进一步由 Mdata 构建 SF，其中 MS 需要使用 SSR, SSR 通过 SSRB 通过传入配置文件调用 configure 来 build)：

```java
sessionFactory = new MetadataSources(
    new StandardServiceRegistryBuilder()
        .configure("hibernate.cfg.xml")
        .build())
    .buildMetadata()
    .buildSessionFactory();
```

**Session API:**

Session 从 Factory 获取数据库信息，然后连接 JDBC，操作数据库。Session 是应用程序和数据库操作的核心，单线程存在。其生命周期很短，在flush之前，所有持久层数据存放在此对象中。相当于 JDBC 的 Connection。
- 获取：get() load()
- 更新：update() save() saveOrUpdate() delete()
- 事务：beginTransaction()
- 管理：isOpen() flush() clear() evict() close()
- Session 提供数据库操纵方法。Session 类似于 JDBC 的 connection 对象。

**Transaction API：**

事务概念，表示一次原子操作。推荐读、写都使用事务。如果不通过事务，不能写入数据到数据库中。提供 commit() rollback() wasCommitted()
等方法


## Hiberante 测试用例和实际入口

Hibernate 的一个测试用例如下：

```java
public class NewsTest {
    private SessionFactory sessionFactory;
    //实际使用不能作为成员变量，有并发的问题
    private Session session;
    private Transaction transaction;
    @Before
    public void init() {
        Configuration configuration = new Configuration().configure();
        sessionFactory = configuration.buildSessionFactory();
        session = sessionFactory.openSession();
        transaction = session.beginTransaction();
    }
    @After
    public void destroy() {
        transaction.commit();
        session.close();
        sessionFactory.close();
    }
    @Test
    public void go() {...}
}
```

注意，在实际情况下，session 和 transaction 不能作为成员变量，有并发问题。此外，实际情况下的 SF 通常和线程绑定，采用如下方式获得 session：

```java
public class HibernateUtils {

    private static HibernateUtils instance = new HibernateUtils();
    private SessionFactory sessionFactory;
    private Session session;

    public static HibernateUtils getInstance() { return instance;}

    private HibernateUtils() {}

    public SessionFactory getSessionFactory() {
        if (sessionFactory == null) {
            Configuration configuration = new Configuration().configure();
            sessionFactory = configuration.buildSessionFactory();
        }
        return sessionFactory;
    }

    public Session getSession() {
        return getSessionFactory().getCurrentSession();
    }
}
```

`getCurrentSession` 返回和线程绑定的 Session，这种方式和 Service 层解除了耦合。当使用 thread 管理的时候，不用关 session，只用提交事务，session 会自动被处理（当事务提交的时候，会自动关闭 session 对象，如果事务被回滚的时候，则自动关闭 session 对象，当关闭后，会返回下一个 session）。注意，使用这种方法必须开启基于线程的 session 获取方式。

注意，基于线程的方式是对 session 而言的，我们仍然需要将 SessionFactory 放置在 Spring IOC 中（配置为 bean），然后注入到 HibernateUtils 中使用（或者在某个地方先执行一次 getSession 获取单例的 SF 对象）。

# Hibernate 基础概念

## Session 缓存操作

Session 是 Hibernate 最常用的一个接口，其主要用于数据库操纵，比如CRUDL。其具有一个缓存，保存着一些 Java POJO。缓存的意义在于，多次相同的查询，可以直接从缓存获得对象，而不是重复向数据库发送请求：

当进行 get 查询的时候，如果找到后，则向引用变量赋值，同时保存引用到 Session 缓存中。这意味着，当引用失效后，Java对象不会被回收，因此下一次 get 会先从 session 缓存中找到对象，进行复用。这也就是为什么要写 id、hashCode 和 equal 方法的原因了。只要 Session 中的缓存没有结束生命周期，那么这个对象将不会被 gc。

```java
News news = session.get(News.class,1);
News news2 = session.get(News.class,1); //实际只执行了一次查询语句。
```

注意这种“高效”是如何完成的：每进行一次查询，获得的对象有两个引用，其一交给用户，其二保存在缓存中，以备下次查询。这和 CPU 多级缓存类似。

缓存操作有三：`flush() reflesh() clear()` 其中 flush 缓存向数据库更新， reflesh 用于数据库向缓存更新， clear 用于清空缓存

**flush**

flush 使数据库和 Session 缓存中对象状态保持一致，如下的语句执行了 select 和 update 语句，更新了对象。
     
```java
News news = session.get(News.class,3);
news.setTitle("Again");
```

注意，在调用的时候，并没有真正的进行数据库的 update，在 flush 后才发送到数据库，但此时尚未提交事务，当最后事务提交后，数据才会被更改。

flush 一般不需要我们手动调用，在 trans.commit() 执行过程中已经被调用了。如下：

 【何时调用】在 trans.commit() 方法中，先调用 session 的 flush() 方法，再提交事务
 
 【如何更新】flush 会自动编写并执行 update 语句，但是不会提交事务，因此在事务结束前不能看到数据更改
 
 【一些特例】这意味着，只有调用 commit 才会进行 SQL 语句操作同步更改，并且提交事务。但是，在未提交事务或者显式调用 session.flush() 之前，可能也会进行 flush() 操作：
     - 执行 HQL, QBC 查询，会先进行 flush 操作，得到最新的数据库记录
     - 如果记录的 ID 是底层数据库自增产生的，在调用 save() 方法后，会自动发送 insert 语句（在 commit 的 flush 之前）获取 id
     - 需要注意，发生这些特例的时候，Hibernate 自动 flush，自然丧失了缓存的优势
     
如果一个全新的对象在自增模式下进行 save，则立即 flush，如果是原先存在的对象，则自动在 commit 执行 flush，不用手动 save:

```java
News news = new News(3,"Hello Java","Marvin",new Date());
session.save(news); //自增下强制 flush 获取 OID 

News news2 = session.get(News.class,1);
news2.setTitle("Hi");
session.save(news2); //多次一举
```

**refresh**

refresh 用来更新 POJO，使其和数据库当前记录保持一致：

```java
News news = session.get(News.class,3);
System.out.println(news);
//如果想要此值得最新字段：
session.refresh(news); //可以看到 hibernate 执行了 select 语句。
//但是这里可能得到的不是最新的数据，这是 MySQL 的事务隔离级别设置问题。更改设置即可。
```

**clear**

clear 会清空 session 中的缓存：

```java
News news = session.get(News.class,3);
session.clear(); 
//清除缓存后，会重新执行查询语句（对于此情况，执行了两条查询）
News news2 = session.get(News.class,3);
```

## Hibernate 对象的状态

根据是否具有 OID，是否存在于缓存、数据表中，区分了四种不同的对象，其中持久化对象和游离对象具有 OID，属于在编，一个在工作，一个在休假。而临时对象和删除对象没有 OID，要么处于入职前的状态，要么处于离职后的状态。

- 持久化对象 - 有 oid，存在于 缓存和数据表中（工作的正式成员）
- 游离对象 - 有 oid，存在于数据表中，但是不存在缓存中（非工作正式成员）
- 临时对象 - 无 oid，不存在于 缓存和数据表中（非正式员工，尚未参与工作）
- 删除对象 - 无 oid，不存在于 缓存和数据库中。

其工作流程如下：
     
起点：一个 POJO 通过 new 变成一个临时对象，此对象没有 oid 和 数据库对应，不存在缓存中。或者通过 get 和 load 直接变成持久化对象。

- 临时对象：一个临时对象通过 save/persist/saveOrUpdate/merge 变成持久化对象，此时具有了 oid，具有缓存和数据库记录。如果没有引用，且尚未变成持久化对象，则进行垃圾回收。

- 持久化对象：一个持久化状态对象通过 evict/close/clear 变成游离状态，此时的 oid 依旧存在，数据库记录也存在，但是不存在于缓存中。或者直接通过 delete 进行删除，编程删除对象。

- 游离对象：一个游离对象通过 update、saveOrUpdate、merge 变成 持久化对象，通过 delete 变成删除对象。

- 删除对象：删除对象直接被 gc。

**临时状态的 save, persist 方法**

save 方法将临时对象变为持久化对象。

<u>save 的 OID 确认问题：</u> 因为临时对象没有 oid，因此 save 方法会为临时对象分配 oid。（通过 insert 语句返回的 id 作为 oid，此操作不在 commit 的 flush 中进行）。在 save 之前设置 id 无效，因为 id 需要执行 insert 和 数据库协商（自增）来确定。在 save 之后，也就是持久化对象的时候，id 不可修改，否则无法进行数据库和缓存对应。

```java
News news = new News(null,"Learn Python","Wiz",new Date());
news.setId(233333);
session.save(news);
//news.setId(34444); 
```

persist 从原始 Class 变为 临时对象，然后变为 持久化对象。persist 和 save 方法类似，都可以将 POJO 从游离状态变成持久状态。

和 save 方法的区别是： 在 persist 之前不能 setId，区别于 save 直接忽略设置 id 的语句，persist 直接抛出异常。

```java
News news = new News(null,"Learn Python","Wiz",new Date());
//news.setId(666);
session.persist(news);
```

**get, load 方法获得持久化状态**

get 从原始 Class 直接变为持久化对象:

```java
News news = session.get(News.class,3);
System.out.println(news);
```

【当存在记录时】load 和 get 的功能差不多，但是 load 并没有马上执行查询，除非调用了此对象的方法，比如 get 或者 toString 此外。如果在调用引用进行操作前关闭了 session 或者 transaction，如果存在数据，则 get 方法正常使用（因为整个对象都被加载到了本地），如果不存在，get 方法则返回 null。但是 load 可能抛出懒加载异常（代理对象在需要时不能通过 session 填充数据）
 
【不存在记录时】如果在使用前未关闭 session 和 transaction，并且数据库中不存在对应的记录，那么 get 返回 null，而 load 抛出异常。

```java
News news = session.load(News.class,3);
System.out.println(news.getClass().getSimpleName());
```

**游离状态的 update 方法**

如果更新一个持久化对象， 不需要使用 update（默认在事务 commit 会调用 session 的 flush，而 flush 会自动将缓存中的持久化对象保存到数据库中）

如果更新一个游离状态对象（一般是之前的 session 关闭，而新建了一个 session 而造成的游离），则需要使用 update 进行加入缓存（入职）的操作。

注意：如果新开一个 session 和 transaction，无论要更新的游离对象和数据表的记录是否一致，都会发送 update 语句。但是，如果在一个 session 和 transaction 中，如果缓存和记录表一致，调用 update 不发送。换句话说，对于游离对象，update 不知道是否数据变更，因此必须强制发送 update，而对于持久化对象，因为 update 前我们可以比较数据是否变更，因此如果不变更则不发送。

可以通过在 hbm.xml 中设置 `select-before-update = true` 来在更新前强制 select， 如果数据有变更，则 update，否则不 update。这种方式可以避免触发器协同工作问题，但是影响效率（如果对持久化对象操作比对游离对象操作少，且需要和触发器协同工作的情况下可用）

【查无此人】注意：如果在数据表中不存在对象记录，则 update 会抛异常（即 update 只用于有 oid 的游离和持久化对象，不适用于临时对象）

【真假冲突】注意：如果使用 update 更新一个对象（来自于游离对象），而 session 中也存在一个 oid 相同的对象（来自于持久化对象），则更新抛出异常。

```java
//对于持久化对象，不需要 update
//News news = session.get(News.class,3);
//news.setAuthor("Marvin");

//对于持久化对象，如果 update，并且没有更改（相比缓存），则不触发 update
//News news = session.get(News.class,3);
//session.update(news);

//对于游离状态（有 oid，但是没有在缓存中，则需要手动 update 来更新游离对象（入职））
News news = session.get(News.class,3);
transaction.commit();
session.close();

session = sessionFactory.openSession();
transaction = session.beginTransaction();
news.setAuthor("Corkine");
session.update(news);
```

**使用 saveOrUpdate 更新游离或临时对象**

update 不能处理临时对象，saveOrUpdate 如果是临时对象，调用 save，如果是游离对象，则调用 update 通过设置一个数据库具有的 oid 可以将临时对象变成游离对象，或者来自其余 session 的有 oid 的对象也是游离对象。

对于临时对象还是游离对象的判断标准在于是否具有 OID。

即：如果没有 oid，则认为是 临时对象，调用 save。如果有 oid（不论是手动设置还是本身就有），则认为是 游离对象，调用 update。和 update 方法一样，如果不存在对应 oid 的数据库记录，则抛出异常。

但是，有时候可能需要为 临时对象添加一个 OID 标识，在这种情况下，可以单独设置 id 的 unsaved-value 属性，则如果有 oid，且 oid 和此值对应，则认为是游离对象，进行 save 而不是 update。之后此 OID 标识会被清除，更换为新的 OID。

```java
News news = new News(null,"Mars","Marvin",new Date());
news.setId(3); //通过为临时对象设置 oid，现在 news 表示的是一个游离对象(除非 oid 等于 unsaved-value)
session.saveOrUpdate(news);
```

**使用 delete 删除持久化或游离对象**

delete 用于删除具有 oid 的游离对象或者持久化对象。

只要 oid 和数据表中一条记录对应，就会准备执行 delete 操作。但是如果不存在此 oid 对应值，则抛出异常。 

delete 在 commit 和 flush 时执行。通过 `hibernate.use_identifier_rollback` 设置为 true，让 delete 后对象 oid 变为 null。在执行 delete 后(尚未提交前)即改变。

```java
@Test public void delete() {
    //对于持久化对象
    News news = session.get(News.class,4);
    session.delete(news);
    //对于游离对象
    News news1 = new News();
    news1.setId(5);
    session.delete(news1);
}
```

**使用 evict 将持久化对象变成游离对象**

evict 从缓存中将指定的持久化对象从缓存中删除，变为游离对象。注意，游离化之后，将从缓存中移除，这时，除非再次进行 update 加入持久化队列，否则将无法进行自动 flush 保存。

```java
@Test public void evict() {
    News news = session.get(News.class,4);
    News news2 = session.get(News.class,7);

    news.setTitle("cccc");
    news2.setTitle("bbbb");
    session.evict(news); //从缓存中移除，则无法进行 flush（flush更新缓存到数据库） 更新此对象
}
```

总结： 一个 class 类可以通过 get/load 来直接得到持久对象，可以通过变成 POJO(临时对象) 再通过 save/saveOrUpdate/persist 变成持久对象。临时对象可以通过SETID变成游离对象，或者持久对象通过 close、clear、evict 得到游离对象，游离对象通过 update/saveOrUpdate 变成持久对象。持久对象和游离对象通过 delete 变成删除对象。


换句话说，一个 class 有以下途径变成持久化对象：

1、【新建】get/load 方法直接获取持久对象
2、【获取】构造 POJO 创建临时对象，然后通过 save、saveOrUpdate、persist方法保存为持久对象
3、【伪造】构造 POJO 创建临时对象，然后通过设置已知的 oid 变成游离对象(不推荐，如果设置的 oid存在的话，则为游离对象，如果设置的 oid 不存在的话，则还是临时对象，因为不能判断是否存在，因此最好不要手动设置 oid，否则就不知道到底这个设置过 oid 的对象是游离对象还是一个临时对象)，游离对象通过 update、saveOrUpdate 变成持久化对象（这种方法类似伪造带薪休假的员工的证明）

此外，持久化对象通过 delete 变成删除对象，通过 close、clear、evict 变成游离对象。游离对象通过 delete 变成删除对象，通过 update、saveOrUpdate 变成持久化对象。

## 执行 SQL 语句以及调用存储过程

调用 Work 接口的 doWork 方法即可调用 SQL 以及存储过程：

```java
Work work = new Work() {
    @Override
    public void execute(Connection connection) throws SQLException {
        PreparedStatement statement = connection.prepareStatement("select * fro username");
        ResultSet set = statement.executeQuery();
        System.out.println("set = " + set);
    }
};
Work work_l = connection -> connection.commit();
```

# Hibernate 对象关系映射

Hiberante 区别于 JdbcTemplate、DBUtils 等工具的一点就是，Hibernate 允许指定各种复杂的映射关系。ORM域模型和数据模型之间的对应可能不是直接的，比如一个类具有另一个类的属性字段，或者有一个集合类，这时候，如何处理类的粒度以完成映射就是一个棘手的问题了。

在之前的流程阶段，我们已经介绍了一个纯粹包含基本（以及受支持）的类型属性的 POJO 的映射以及其主要的参数，在这里则主要介绍容器属性、复杂类属性的映射。Hibernate Mapping 提供了对于复杂关系映射解决方案的支持，比如一对多、多对多、单向、双向、集合类等等。

## 组成关系

```java
public class Worker {
    private String name;
    private Integer id;
    private Pay pay;
    ...
}
public class Pay {
    private int monthlyPay;
    private int yearPay;
    private int vocationWithPay;
    ...
}
```
```xml
<class name="Worker" table="worker">
    <id name="id" type="java.lang.Integer" column="id">
        <generator class="native"/>
    </id>
    <property name="name" type="java.lang.String" column="name" />
    <!--hibernate 的映射分为 value 类型 和 entity 类型。其中后者无法使用 property 标签声明映射，必须通过
    component 进行声明-->
    <component name="pay">
        <property name="monthlyPay" column="monthly_pay" />
        <property name="yearPay" column="year_pay" />
        <property name="vocationWithPay" column="vocation_with_pay" />
    </component>

</class>
```

组成关系指的是，一个字段需要映射一个类，而此类并不对应一个其他的表，然后不使用数据库字段使用外键进行连接，而是直接保存此类到此表中，称之为组合。组合不能使用 property 标签，需要使用 component 标签，此标签中的 name 属性为 pojo 引用名称， property 子标签负责将数据库中的列对应为此属性的类的字段上。提供 name 对应此类属性的 pojo 引用，提供 column 提供此类属性对应的数据库列。


## 单向一对多关联关系

一对多的关联关系是区分方向的，称一和多端。其中单向的o2n的 o端不需要知道 n端，但是必须可以从 n端访问 o段。也就是说，n 端必须提供一个 o端的引用。单向一对多一句话概括为：偶像和粉丝的关系。


一个简单的 单向 o2n 如下：

```java
public class Customer {
    private String customerName;
    private Integer customerId;
    ...
}
public class Order {
    private Integer orderId;
    private String orderName;
    private Customer customer;
    ...
}
```

注意，在这个例子中， Order 类是 n 端，其包含一个指向 o 端的引用。其对象映射文件如下：

```xml
<class name="Customer" table="customers">
    <id name="customerId" type="java.lang.Integer" column="customer_id">
        <generator class="native"/>
    </id>
    <property name="customerName" type="string" column="customer_name" />
</class>
<class name="Order" table="orders">
    <id name="orderId" type="java.lang.Integer" column="order_id" >
        <generator class="native"/>
    </id>
    <property name="orderName" type="string" column="order_name" />
    <many-to-one name="customer" class="Customer" column="customer_id" />
</class>
```

单向关系，对于表而言，o 端的表不需要包含 n 端的信息。但是 n 端的表必须包含 o 端的外键，使用 many-to-one 标签指定此外键的 Java 类属性（name），Java 类型（class）以及对应的列名（column）。

可选参数为 cascade，lazy，fetch。

【增】

增加对象有顺序上的效率问题：

```java
Customer customer = new Customer();
customer.setCustomerName("Liujin");
Order order1 =  new Order();
order1.setOrderName("order-3");
order1.setCustomer(customer);
Order order2 =  new Order();
order2.setOrderName("order-4");
order2.setCustomer(customer);

session.save(order1);
session.save(order2);
session.save(customer);
```

如果不设置 customer 的 id，则一共五条语句，因为先插入 order 需要 customer 外键，而当前 customer 尚未保存。因此先留空，然后执行sql语句。之后当 customer 保存后又进行 update 更新外键，因此有 insert 和 update 语句。

如果设置 customer 的 id，则插入失败：Cannot add or update a child row: a foreign key constraint fails。因为保存此 order 需要 customer 的 oid，如果保存的属性是一个对象，但是这个对象尚不存在，那么留空，但是，如果这个对象提供了oid，那么就找到对应的oid即可。

如果先进行 o端的保存，再进行 n端的保存，则只有 insert 语句，这样效率最高。

【删】

```java
Customer customer = session.get(Customer.class,2);
session.delete(customer);
```

如果删除的是 o端，而o有一些引用的 n端对象，则不能删除 o端，必须先清空 n端。n端可以正常删除。

【改】

```java
Order order = session.get(Order.class,1);
order.getCustomer().setCustomerName("Marvin");
```
【查】

```java
Order order = session.get(Order.class,1);
System.out.println(order.getOrderName()); //order 是一个代理对象
//session.close(); 懒加载异常触发
System.out.println(order); //执行了第二次 select 查询
```

查询 o端，没有问题。查询 n端，则默认使用 o的代理对象填充 pojo 属性。当 session 关闭，则触发懒加载异常，使用 lazy = false 关闭懒加载。


## 双向一对多关联关系

如下：

```java
public class Customer {
    private String customerName;
    private Integer customerId;
    private Set<Order> orders = new HashSet<>();
    ...
}
public class Order {
    private Integer orderId;
    private String orderName;
    private Customer customer;
    ...
}
```

双向一对多关系类似于订阅者和发布者关系，发布者有订阅者的引用，而订阅者也有发布者的引用，但是，发布者的引用是集合，而订阅者则是一个类。

注意，o 端的集合最好采用接口集合类型，此外，必须进行初始化，防止空指针和代理包装后的懒加载问题。


```xml
<class name="Customer" table="customers">
    <id name="customerId" type="java.lang.Integer" column="customer_id">
        <generator class="native"/>
    </id>
    <property name="customerName" type="string" column="customer_name" />
    <set name="orders" table="orders" inverse="true" cascade="delete-orphan" order-by="order_name desc">
        <key column="customer_id"></key>
        <one-to-many class="Order"/>
    </set>
</class>
<class name="Order" table="orders">
    <id name="orderId" type="java.lang.Integer" column="order_id" >
        <generator class="native"/>
    </id>
    <property name="orderName" type="string" column="order_name" />
    <many-to-one name="customer" class="Customer" column="customer_id" />
</class>
```

双向 n2o 的关系中，对于 n 的一端，同样需要指定 many-to-one 标签，规定类、类引用名称、列名称。此外，对于 o 的一段，需要使用 set 来指定集合，set 标签需要指定 name 属性（对应POJO set 引用的名称），table 属性（对应存放集合元素的表）。

注意，双向一对多的多端的集合信息其实是由一端的表中的一个字段维护的。

使用 set 标签的子标签来说明此字段如何和表对应的，需要提供容器泛型的 Java 类以及保存在它表中的能够查到自己的那个列名，具体而言：key 子标签，其中的 column 属性对应它表中的自己的列名，此列名保存了 o 端的 OID。one-to-many 子标签，规定了此列名对应的容器泛型的 Java 类。

其余有用的属性，比如双向关系维护人，删除、保存、排序策略：

- 通过 inverse 属性为 true 来让 parent 不维护关联关系，而让 son 维护关系


- 通过 cascade 属性更改删除和保存策略。实际情况下推荐手工进行删除和保存，不使用级联。
    - 设置为 delete 可以从 1 的一端删除其对象，包括所有有其引用的对象（删除1端的对象）
    - 设置为 delete-orphan 可以删除孤儿（删除n端的某些对象）
    - 设置为 cascade="save-update" 可以级联保存。
    
    
- 通过 order-by 设定集合排序的方式，需要指定排序需要的名称和排序方式，使用的是列名。

**使用须知**

在保存时，先保存 o端，再保存 n端执行的 SQL 最少。因为双向 n2o 两面都要维护和对方的关系，因此通过 inverse = true 来放弃 o 端的控制权。降低查询量。

在加载 o端的时候，默认情况下，使用了 lazy = true 的懒加载，因此，关闭 session 即出现获取集合元素异常。可以关闭懒加载以提取全部信息，这时候在关闭 session 后引用仍然可以正常工作。

【增】

设置 cascade="save-update" 可以级联保存

```java
Customer customer = new Customer();
customer.setCustomerName("Corkine");
Order order1 =  new Order();
order1.setOrderName("order-1");
order1.setCustomer(customer);
Order order2 =  new Order();
order2.setOrderName("order-2");
order2.setCustomer(customer);

customer.getOrders().add(order1);
customer.getOrders().add(order2);

session.save(customer);
```
【删】

双向关系无法简单进行 delete，需要指定 cascade = "delete" 级联删除（在删除1的一端的时候，删除多的一端）
  
```java
Customer customer = session.get(Customer.class,5);
session.delete(customer); //失败，因为包含外部引用，除非指定 cascade 设置为 delete
Customer customer = session.get(Customer.class,4);
customer.getOrders().clear(); //失败，只有 select 操作，并不会进行删除，除非在 cascade 设置 delete-orphan（允许删除孤儿）
```
【改】

可以从 o端的集合中直接获取 n端对象，并对其进行更改：

```java
Customer customer = session.get(Customer.class,4);
customer.getOrders().iterator().next().setOrderName("Order_no_exist");
```
【查】

```java
Customer customer = session.get(Customer.class,3);
System.out.println(customer.getCustomerName());
//session.close(); 同样的懒加载，关闭 session 后出现异常
System.out.println(customer.getOrders().getClass());
//class org.hibernate.collection.internal.PersistentSet
```

## 基于外键映射的一对一关系

```java
public class Department {
    private Integer depId;
    private String depName;
    private Manager mgr;
    ...
}
public class Manager {
    private Integer mgrId;
    private String mgrName;
    private Department dep;
    ...
}
```

一对一关系中，互相有对方的引用，对于数据库而言，难点就是如何保证两张表的对应。很显然，两张表均具有对照难以维护，因此并不完全映射字段到类，考虑将某一端的主键放置到另一端：在主端增加一个外键，并且限制此外键为 unique，表示 o2o 关联，然后在从端，不提供含有对方引用的列，而是通过对另外一张表查找外键来确定从端POJO的此类属性）。这种模式称之为基于外键的o2o，即一端在表中放弃对方引用，而另一端在表中增加外键以及引用限制。

```xml
<class name="Department" table="DEPARTMENT">
    <id name="depId" column="DEPT_ID" type="integer" >
        <generator class="native" />
    </id>
    <property name="depName" column="DEPT_NAME" type="string" />
    <!--基于外键的 one-one 映射，带有外键的一端可以看作 many-to-one 的 unique 版本。-->
    <many-to-one name="mgr" class="Manager" column="MANAGER_ID" unique="true" />
</class>
<class name="Manager" table="MANAGER">
    <id name="mgrId" column="MGR_ID" type="integer">
        <generator class="native" />
    </id>
    <property name="mgrName" type="string" column="MGR_NAME" />
    <!--如果没有 property-ref 属性，则默认使用两者的主键进行对应，否则使用 property-ref 进行对应-->
    <one-to-one name="dep" class="Department" property-ref="mgr"/>
    <!--注意，此处不能使用 many-to-one，即不能让 Manager 同样持有 Department 的主键，否则不是真正的一对一：
    A1,A2 两男， B1,B2 两女，A1 喜欢 B1，但是B1 喜欢 A2，同样 A2 喜欢 B2， 但是 B2 喜欢 A1，这也是一一对应，但
    不是我们期望的一对一-->
</class>
```

注意，基于外键的 o2o 类似于双向 n2o（订阅者端），对于带有外键映射的表而言，需要使用 many-to-one 标签，提供 pojo 引用、Java 类型、列名，此外最重要的，限制此外键唯一。

对于 o2o 的从端（不维护外键的这一侧）提供 one-to-one 标签，提供引用的类、pojo 属性名，此外，还要提供 property-ref 需要对应的它表的列名。只有这样，才能够从另一张表中找到此 pojo 需要字段的引用对象。（除非这两个表共用一个 OID 列？）

在具体的使用上，基于主键的 o2o 的增加操作建议先保存没有外键列的对象，这样消耗的资源少。（即总是先保存较弱的一方）。在查找操作时，在查找具有外键的一方时，可能有懒加载的问题，但是在查询没有外键的一方时，回自动进行关联对象的查询。此外， one-to-one 需要指定自己的引用在对方表上的列明，否则将会使用对方的主索引列进行对应查找。

```java
@Test public void save() {
    Department department = new Department();
    department.setDepName("Dept-a");
    Manager manager = new Manager();
    manager.setMgrName("Mgr-a");

    department.setMgr(manager);
    manager.setDep(department);

    //建议先保存没有外键列的对象，这样消耗资源较少
    session.save(manager);
    session.save(department);
}

@Test public void get() {
    //在查询具有外键的对象是，默认没有进行其对应关系的表的查询（Manager）而是使用了懒加载
    Department department = session.get(Department.class,1);
    System.out.println(department.getDeptName());
    //同样的，懒加载问题，可能触发懒加载异常问题
    //o2o 需要指定 ref 查询对应：DEPARTMENT department1_on manager0_.MGR_ID=department1_.MANAGER_ID
    System.out.println(department.getMgr().getMgrName());
}

@Test public void get2() {
    //在查询没有外键的对象时，默认进行了外键关联对象的查询，可以看到这里没有使用懒加载
    Manager manager = session.get(Manager.class,1);
    System.out.println(manager.getDep().getDepName());
}
```

总的而言，many-to-one 标签指的是含有一个列，此列存放着其余表的外键，此属性存放着一个对应着其他表的对象这种情况，可以使用在单/双向多对一的多的一端，也可以使用在基于外键的一对一的主端（必须添加唯一性约束）。one-to-many 标签用在双向多对一的一端的集合类中，但是却被用在这种类型的多的一端的表中，表示这些行所属的其余表中的行。one-to-one 标签被用在基于外键的一对一的从属端，表示没有此对应列，要找内容需要去另一个表查找。

## 基于主键映射的一对一关系

```java
<class name="Department" table="DEPARTMENT">
    <id name="depId" column="DEPT_ID" type="integer" >
        <generator class="foreign">
            <!--另一端参照 mgr 属性的主键创建 id , 同时要求启用 constrained 以开启主键约束-->
            <param name="property">mgr</param>
        </generator>
    </id>
    <property name="depName" column="DEPT_NAME" type="string" />
    <one-to-one name="mgr" class="Manager" constrained="true" />
</class>
<class name="Manager" table="MANAGER">
    <id name="mgrId" column="MGR_ID" type="integer">
        <generator class="native" />
    </id>
    <property name="mgrName" type="string" column="MGR_NAME" />
    <!--一端采用 one-to-one 设置关联属性，id 自增-->
    <one-to-one name="dep" class="Department" />
</class>
```

这种方式指其中的一端主键生成采用 foreign 策略，表明根据对方主键生成自己主键。需要对 generator 采用 foreign 类，并在 param 中定义好需要参照创建主键的 POJO 引用的名称。在另一端，和之间一样，提供 one-to-one 的标签，表明此字段的类以及 pojo 引用名，Hibernate 会自动到 Department 类对应的表中查找对应经理的部门。

这种方式的冗余更小。需要注意，这里的 one-to-one 因为不是查找外键，因此不能定义 param-ref 属性。

使用如下，插入顺序没有影响，因为没有主键无法保存，因此必须是从端的那一个先保存。此外，获取具有外键的对象时，可能有懒加载，获取没有外键的对象是，直接进行了关联查询，这和基于外键的一对一对应类似。

```java
@Test public void save() {
    Department department = new Department();
    department.setDepName("Dept-b");
    Manager manager = new Manager();
    manager.setMgrName("Mgr-b");

    //插入顺序没有影响，主键不能为空，所以，保存 dep 的时候必定要先保存 manager 获取主键才能保存。
    //因此顺序没有影响。此外，不能使用 property-ref，因为这里没有需要对应主键和外键的地方，仅仅是两个主键对应。
    manager.setDep(department);
    department.setMgr(manager);

    session.save(manager);
    session.save(department);
}

@Test public void get() {
    //在查询具有外键的对象是，默认没有进行其对应关系的表的查询（Manager）而是使用了懒加载
    Department department = session.get(Department.class,1);
    System.out.println(department.getDeptName());
    //同样的，懒加载问题，可能触发懒加载异常问题
    //o2o 需要指定 ref 查询对应：DEPARTMENT department1_on manager0_.MGR_ID=department1_.MANAGER_ID
    System.out.println(department.getMgr().getMgrName());
}

@Test public void get2() {
    //在查询没有外键的对象时，默认进行了外键关联对象的查询，可以看到这里没有使用懒加载
    Manager manager = session.get(Manager.class,1);
    System.out.println(manager.getDep().getDepName());
}
```

## 单向多对多映射关系

单向的多对多映射必须使用连接表，在表中规定两者的对应关系。对于 POJO 而言，其中的一方含有对方的 set，而另一方则没有关于对方的信息。

这里将 Category 中的 items 属性删除即为单向多对多关系，保留则为双向多对多关系。

```java
public class Category {
    private Integer id;
    private String name;
    private Set<Item> items = new HashSet<>();
    ...
}
public class Item {
    private Integer id;
    private String name;
    private Set<Category> categories = new HashSet<>();
    ...
}
```

对于单向多对多而言，对于不知道对方的那一端，不需要提供 set。下面以双向多对多为例：

```xml
<class name="Item" table="ITEMS">
    <id name="id" column="ITEM_ID">
        <generator class="native"/>
    </id>
    <property name="name" type="string" column="ITEM_NAME"/>
    <!--单向多对多不需要写此处的 set, 双向多对多需要设置 inverse，避免重复主键-->
    <set name="categories" table="CATES_ITEMS" inverse="true">
        <key column="ITEM_ID"></key>
        <many-to-many class="Category" column="CATE_ID" />
    </set>
</class>
<class name="Category" table="CATEGORY">
    <id name="id" column="CATE_ID">
        <generator class="native"/>
    </id>
    <property name="name" column="CATE_NAME" type="string" />
    <!--单向多对多需要指定中间表，因此需要使用 table 指定此表
    many-to-many 的 class 指定多对多关系的持久化类型， column 指定在中间表中持久化类的外键名称-->
    <set name="items" table="CATES_ITEMS">
        <key column="CATE_ID"></key>
        <many-to-many class="Item" column="ITEM_ID" />
    </set>
</class>
```

双向多对多和双向一对多的多段类似，需要使用 set 标签，提供 name 对应 pojo 属性，提供 table 定义中间表。在子标签中，使用 key 标签定义中间表的自己的字段，使用 many-to-many 标签定义集合内类型的类以及其在中间表的对应字段。

对于对方的类，也需要做相同设置，定义 pojo 引用，中间表名，中间表的自己的列名，中间表的容器泛型类的列名和容器泛型类的类型。

但是，需要注意，inverse 必须在某一段设置为 true，以将控制权交给对方。

此外需要注意，和 many-to-one 对比，many-to-many 中也提供了 column 属性，这个属性应该和 key 中的 column 属性进行区分，key 中的属性是在中间表中的自己的列名，many-to-many 中的属性是在中间表中别人的列名。在 many-to-one 中，这里不需要提供列名，是因为我们直接找到对方的类表，对方的类表保留了我们的字段，能够进行映射，找到对应行，而这里找到是中间表，然后从中间表的某一列找到对方的类表对应的行，然后到对方那里直接按主键找到取出。

## 继承映射关系：subclass

```java
public class Person {
    private Integer id;
    private String name;
    private Integer age;
    ...
}
public class Student extends Person {
    private String school;
    ...
}
```

```xml
<class name="Person" table="PERSONS" discriminator-value="PERSON">
    <id name="id" column="PERSON_ID">
        <generator class="native"/>
    </id>

    <discriminator column="TYPE" type="string"/>

    <property name="name" column="NAME" type="string"/>
    <property name="age" column="AGE" type="integer" />

    <!--使用 subclass 进行映射，需要在同一个 class 下声明 subclass 独有的 property pojo 和 sql 字段和类型对应
    此外，还要启用辨别者列（discriminator 标签）、声明父类和子类的各自的辨别者列名称（discriminator-value属性）-->
    <subclass name="Student" discriminator-value="STUDENT">
        <property name="school" type="string" column="SCHOOL"/>
    </subclass>
</class>
```

基于 subclass 的继承映射很简单，使用一张表，使用 subclass 标签提供对于子类的支持，在此标签内部使用 property 指定子类独有的字段。需要启用 discriminator 来指定辨别者列的列名和类型。对于子类，在 subclass 标签中应该指定 discriminator-value 以指定辨别者列，对于父类，应该在 class 处提供 discriminator-value 提供辨别者列的父类类名。

使用指南：

【增】

书韩剧会增加到一张表上，子类独有的字段放置在独有的列中。这种方式的限制是：这些子类独有的列不能限制为空，否则如果是其他类型，则无法进行插入。

```java
@Test public void save() {
    Person person = new Person();
    person.setAge(20);
    person.setName("Marvin");
    session.save(person);

    Student student = new Student();
    student.setAge(22);;
    student.setName("Liujin");
    student.setSchool("CCNU");
    session.save(student);
}
```

【查】

使用 subclass 查询父类或者子类，只需要查询一次（使用辨别者列）。但是缺点是

1、使用了辨别者列，增加了冗余
2、子类独有字段不能添加非空约束
3、如果继承层次较深，那么数据表字段较多

```java
@Test public void query() {
    List<Person> people = session.createQuery("from Person ").list();
    System.out.println(people.size());

    List<Student> students = session.createQuery("from Student ").list();
    System.out.println(students.size());
}
```

## 继承映射关系：joined-subclass

这种方式的特征是：

- 每个子类一张表
- 父类由父类表维护，子类由父类和子类表共同维护。共有属性保存在父类表中，子类独有属性保存在子类表中。

这种方式的优点是：

- 没有鉴别者列，但是需要设置子类映射共有主键。
- 子类可以添加非空约束。

使用方法：

```xml
<class name="Person" table="PERSON">
    <id name="id" column="PERSON_ID">
        <generator class="native"/>
    </id>

    <property name="name" column="NAME" type="string"/>
    <property name="age" column="AGE" type="integer" />

    <joined-subclass name="Student" table="STUDENT">
        <key column="PERSON_ID"></key>
        <property name="school" column="SCHOOL" type="string"/>
    </joined-subclass>
</class>
```

需要在父类中启用 joined-subclass 标签，在其中声明子类对应的表（name-table）。采用 property 子标签提供子类独有的字段。因为子类的属性需要使用子类表+父类表获得，因此使用 key 子标签提供在父类表中查找子类共有属性的比较列（column 定义的列即是主键，又是外键）。

这种方式的缺点是： 查询弱、保存弱，都需要处理多张表，SQL 语句多。但因为其避免了 subclass 的子类非空限制，因此使用较多。


## 继承映射关系：union-subclass

这种方式的 xml 文件如下：

```xml
<class name="Person" table="PERSONS">
    <id name="id" column="PERSON_ID">
        <generator class="increment"/>
    </id>

    <property name="name" column="NAME" type="string"/>
    <property name="age" column="AGE" type="integer" />
    <!--union-subclass 不用声明辨别者列，只用声明子类的表，以及独有属性的映射。-->
    <union-subclass name="Student" table="STUDENTS" >
        <property name="school" type="string" column="SCHOOL" />
    </union-subclass>
</class>
```

需要提供 union-subclass 标签，在其中应该声明子类的表、子类独有的属性映射。这种方式下，子类所有的数据均保存在子类自己的表中，而不保存在父类的表中，因此对于主键没有限制。这种方式避免了子类的非空约束，也不需要鉴别者列，缺点是，冗余数据较多。

这种方式的第二个限制是，主键的生成方式不能是 identity，也不能是 native，因为同一个类继承层次中的所有实体类都应该使用同一个主键种子。而是用默认的数据库 identity/sequence，那么造成记录的主键不连续。

这种方式的保存性能不错，查询子类也可以。但是在查询父类时，需要挨个遍历子类的表，性能偏弱。

# Hiberante 检索策略

## 类级别的检索策略

使用 class 标签的 lazy 属性来指定类的检索策略。可选 true、false。如果加载一个对象希望使用其属性，则立即检索，如果只是希望获得其引用，则使用延迟检索。

需要注意，不论 lazy 设置如何，Session 的 get 和 Query 的 list 方法将始终采用立即/延迟检索，而 load 方法则会在 lazy = true 的时候进行懒加载，而在 lazy 设置为 false 的情况下不进行懒加载（lazy 仅对 load 有效）

## 一对多和多对多检索策略

集合级别的检索策略由 lazy 和 fetch 控制，其中 lazy 可选在获得此对象即初始化，还是在使用集合时才初始化，默认为 true。fetch 可选 select、subselect、join。前两种指的是不同的 SQL 语句执行形式，join 指的是使用迫切左外连接策略进行查询。默认为 select。

lazy 除了立即、延迟，还可以选择增强延时，当对集合获取 size、isEmpty 等信息，依旧不进行检索，只当调用 iterator 才去检索。

batch-size 可以设置每次获取的数据个数。

fetch 可选 select、subselect 和 join。其中迫切左外连接将会自动忽略 lazy 属性值。

## 多对一和一对一检索策略

lazy 可选 proxy，no-proxy, false。其中 proxy 指的是延迟，no-proxy 指的是无代理延迟，false 指的是立即。

fetch 同上，选择 join 将忽略 lazy 属性。

## 检索策略总结

get、load、list 是三种检索方式，get 只能立即获取，list 只能延迟加载（除了 HQL 手动指定 fetch-join），其中对于配置，只对 load 有效。

此外，配置分为 lazy 和 fetch 两级。不论 lazy 如何设置，fetch 设置为 join，则将忽略 lazy，对 load 使用迫切左外连接模式。如果 fetch 不使用 join， 则按照 lazy 的设置来。


# Hibernate 检索方式

Hibernate 可以通过导航、OID、HQL、QBC、SQL 的方式进行检索。对于 HQL 而言，其是一个面向对象的 SQL 封装，使用最广。其支持各种查询条件、投影查询、分页查询、连接查询、分组查询、子查询、支持内置函数、动态绑定参数。其使用方法如下：

## 使用 HQL 进行查询

**进行查询**

获得 Query 对象(session.createQuery)，填充占位符(Query.setParameter)，进行查询(Query.list)：

```java
//创建 Query 对象（createQuery（sql））
String hql = "from Employee e where e.salary > ?1 and e.email like ?2 and e.dept = ?3" +
        "order by e.id";
Query query = session.createQuery(hql);
//填充占位符（setParameter）
Department department = new Department(); department.setId(233);
query.setParameter(1,100.0).setParameter(2,"%%").setParameter(3,department);
//进行查询（list）
List<Employee> employees = query.list();
System.out.println(employees);
```

**进行更新**

```java
String hql = "delete from Department d where d.id = :id";
Query query = session.createQuery(hql);
query.setParameter("id",100).executeUpdate();
```

**分页查询**

使用 setMaxResults, setFirstResult 进行限制分页、跳过

```java
String hql = "from Employee e ";
Query query = session.createQuery(hql);

int pageNo = 3;
int pageSize = 5;
//setFirstResult 从0开始设定检索起始位置， setMaxResults 设置最大检索数量
List<Employee> employees =
query.setFirstResult((pageNo - 1) * pageSize).setMaxResults(5).list();

System.out.println(employees);
```

**命名查询**

使用 getNamedQuery 进行命名查询，需要指定此 query 的 name 字段，作为引用创建查询。此 query 应在 cfg 的一个 map 中。注意，使用 `<![CDATA[]]>` 来防止转义。

```xml
<query name="salaryEmps"><![CDATA[FROM Employee e where e.salary > :minSal and e.salary < :maxSal]]></query>
```

```java
Query query = session.getNamedQuery("salaryEmps");
query.setParameter("minSal",100.0).setParameter("maxSal",10000.0);
List<Employee> list = query.list();
System.out.println(list);
```

**属性/投影查询**

```java
String hql = "select e.id, e.name from Employee e where e.dept = :dept";
Query query = session.createQuery(hql);
Department dept = new Department(); dept.setId(1);
List<Object[]> result = query.setParameter("dept",dept).list();
for (Object[] objects : result) {
    System.out.println(Arrays.asList(objects));
}
```

注意，返回的结果是 `Object[]` 的容器，这里面对应的 `Object[]` 是我们查找的两个属性。

```java
String hql = "select new Employee(e.id, e.name) from Employee e where e.dept = :dept";
Query query = session.createQuery(hql);
Department dept = new Department(); dept.setId(1);
List<Employee> result = query.setParameter("dept",dept).list();

System.out.println(result);
```

可以将查找到的属性创建一个新的对象，这时返回的就是此对象的集合了。

**分组查询**

```java
String hql = "select min(e.salary), max(e.salary) from Employee e " +
                "group by e.dept having min(salary) > :minSal";
Query query = session.createQuery(hql);
query.setParameter("minSal",10.0);
List<Object[]> list = query.list();
for (Object[] objects : list) System.out.println(Arrays.asList(objects));
```

分组查询使用 group by 命令，其中 having 提供了分组的条件，可选 count() max() min() sum() avg()。

**(迫切)左外连接**

使用迫切左外连接强制获取集合对象，而不是使用集合元素的代理。

```java
String hql = "select distinct d from Department d left join fetch d.emps";
Query query = session.createQuery(hql);
List<Department> list = query.list();
System.out.println(list.size());
for (Department department : list) {
    System.out.println(department + ", " + department.getEmps().size());
}
```

迫切左外连接使用 `left join fetch` 语法。如果想要去除重复数据，则需要添加 `select distinct d ...` 限定，或者使用 set 过滤重复。迫切左外连接可以立即获取对象，而不使用代理，其优先级最高。

同上，左外连接语法为 `from Department d left join d.emps` 。没有 fetch，将会按照配置文件的方式进行查询，默认是 lazy。

**(迫切)内连接**

同（迫切）左外连接，语法为 `from Department d inner join d.emps` inner 可以省略。区别（迫切）左外连接的地方时，取双方的交集而不是并集。

## 使用 SQL 进行查询

需要注意，SQL 使用的是表名，而 HQL 使用的是 POJO 的类属性名。可以为 SQL 提供别名指定 POJO 类属性名。

```java
String sql = "insert into DEPARTMENTES VALUES(?1,?2)";
Query query = session.createSQLQuery(sql);
query.setParameter(1,100).setParameter(2,"DEP-Spec").executeUpdate();
```

此外，采用 org.hibernate.Worker.doWork 接口方法，也可以写 SQL，也可以在此执行批处理查询。

# Hibernate 二级缓存

除了 Session 的缓存，SessionFactory 也带有缓存。SessionFactory 具有内外缓存，其中映射文件和内置的 SQL 语句位于内部 SF 缓存中，只读不可修改。其外部缓存的数据是数据库数据的复制，外置缓存物理介质可以是内存或者硬盘。

二级缓存不保证并发，适合很少被修改的数据。如果数据与其他应用程序共享、涉及金钱等敏感信息，经常被修改，则不适合放置到二级缓存中。

因为 Hibernate 仅仅定义了二级缓存的标准，但是没有提供实现，因此需要提供实现（jar包/maven依赖）。

二级缓存的启用步骤： 在 cfg.xml 中启用二级缓存、指定实现工厂、指定哪些类启用二级缓存/二级缓存策略是什么。

```xml
<!--启用二级缓存-->
<property name="cache.use_second_level_cache">true</property>
<property name="cache.region.factory_class">org.hibernate.cache.ehcache.internal.EhcacheRegionFactory</property>
<!--配置查询缓存-->
<property name="cache.use_query_cache">true</property>

<class-cache class="com.mazhangjing.hql.Employee" usage="read-write" />
<class-cache class="com.mazhangjing.hql.Department" usage="read-write" />
<collection-cache collection="com.mazhangjing.hql.Department.emps" usage="read-write" />
```

```java
Employee employee = session.get(Employee.class,100);
System.out.println(employee);

transaction.commit();
session.close();

session = sessionFactory.openSession();
transaction = session.beginTransaction();

Employee employee2 = session.get(Employee.class,100);
System.out.println(employee2);
```

如上所示，在 class-cache 中对相应的类启用了二级缓存，当关闭 session 之后，我们再次获取相同对象，并没有执行 SQL 语句，而是从二级缓存中获得了对象。

二级缓存不仅仅能够作用在类级别，还能作用在集合级别，使用 collection-cache 标签设置 collection 和 usage 即可。

```java
//集合级别的二级缓存，需要将类、类的集合、集合中的类均设置二级缓存。
//除了在 cfg.xml 中配置，也可以在 hbm.xml 中的 class 和 set 标签中配置。
Department department = session.get(Department.class, 2);
System.out.println(department.getName());
System.out.println(department.getEmps().size());

transaction.commit();
session.close();

session = sessionFactory.openSession();
transaction = session.beginTransaction();

Department department2 = session.get(Department.class, 2);
System.out.println(department2.getName());
System.out.println(department2.getEmps().size());
```

需要注意，二级缓存对于 HQL 和 QBC 默认无效，需要开启查询缓存：采用 `<property name="cache.use_query_cache">true</property>` 并且 `Query::setCacheable(true)` 来进行带有缓存的查询。

```java
Query query = session.createQuery("from Employee e");
query.setCacheable(true);
List<Employee> list = query.list();
System.out.println(list);
```

对于 ehcache 而言，配置文件在类路径下的 ehcache.xml 下，可以提供各种对于具名（指定类别）的缓存策略配置，以及指定缓存文件夹：

```xml
<!--写入数据到硬盘时，写入到此： d:\\tempDir-->
<diskStore path="user.dir"/>
<defaultCache
        maxElementsInMemory="10000"
        eternal="false"
        timeToIdleSeconds="120"
        timeToLiveSeconds="300"
        overflowToDisk="true"
/>
<!--具名的配置策略，可对于集合进行配置-->
<cache name="com.mazhangjing.hql.Employee"
       maxElementsInMemory="10000"
       eternal="false"
       timeToIdleSeconds="300"
       timeToLiveSeconds="600"
           overflowToDisk="true"
/>
<cache name="com.mazhangjing.hql.Department.empl"
       maxElementsInMemory="1000"
       eternal="true"
       timeToIdleSeconds="0"
       timeToLiveSeconds="0"
       overflowToDisk="false"
/>
```

其中，默认的存储策略 如果没有对应的命名策略，则使用默认策略。缓存具名对于类而言，指的是全类名，对于集合而言，指的是类名+ . +属性名。

- maxElementsInMemory 在内存中最多存储对象数目
- eternal 对象在缓存中是否为永久的
- timeToIdleXXX 最长空闲时间，单位为秒，超过此事件则过期，自动清除。0表示无限空闲状态
- timeToLiveXXX 最长生存时间，单位为秒，必须大于空闲时间（起码要等于）。
- overflowToDisk 内存缓存数目达到上限后，是否将溢出对象设写到基于硬盘的缓存中。